<a href="https://colab.research.google.com/github/jmasonlee/efficiently_testing_etl_pipelines/blob/main/fixing_a_big_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup Notebook

In [2]:
!rm -rf efficiently_testing_etl_pipelines
!git clone https://github.com/jmasonlee/efficiently_testing_etl_pipelines.git
!cp -r /content/efficiently_testing_etl_pipelines/src/ .
!cp -r /content/efficiently_testing_etl_pipelines/tests/ .
!rm -rf efficiently_testing_etl_pipelines
!rm -rf tests/diamond_pricing_test*
!rm -rf tests/test_helpers/*verification_helpers.py
!rm -rf tests/conftest.py
!rm -rf sample_data


Cloning into 'efficiently_testing_etl_pipelines'...
remote: Enumerating objects: 550, done.
remote: Counting objects: 100% (191/191), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 550 (delta 121), reused 111 (delta 70), pack-reused 359
Receiving objects: 100% (550/550), 246.07 KiB | 8.48 MiB/s, done.
Resolving deltas: 100% (327/327), done.


# Setup Tests

### Install Dependencies

For the exercise, we will need some special dependencies to allow us to run lots of tests in a notebook.

`ipytest` lets us run our tests in a notebook.



In [3]:
!pip install ipytest

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.9 MB/s eta 0:00:00


ipytest is what allows us to run our tests in a notebook. This next cell is not needed if you are writing tests in a separate pytest file.

In [4]:
import ipytest
ipytest.autoconfig()

We are installing `pyspark` because it doesn't come with the base colab environment

In [5]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=13b8460b2c01651b31ebdfda3c116e8d7519284d585e49e934055dee2caf3462
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [6]:
!pip install chispa

## Create a local SparkSession

Normally spark runs on a bunch of executors in the cloud. Since we want our tests to be able to run on a single dev machine, we make a fixture that gives us a local spark context.

In [7]:
import pytest
from _pytest.fixtures import FixtureRequest
from pyspark import SparkConf
from pyspark.sql import SparkSession

@pytest.fixture(scope="session")
def spark(request: FixtureRequest):
    conf = (SparkConf()
        .setMaster("local")
        .setAppName("sample_pyspark_testing_starter"))

    spark = SparkSession \
        .builder \
        .config(conf=conf) \
        .getOrCreate()

    request.addfinalizer(lambda: spark.stop())
    return spark

## Create Helpers

This is a helper function that retrieves our test output from the expected.json file

In [8]:
import json

def expected_json():
    with open("tests/fixtures/expected.json") as f:
        return json.loads(f.read())

# The Test

In [16]:
%%ipytest -qq
from src.linear_regression_prep import transform
from tests.test_helpers.json_helpers import create_df_from_json, data_frame_to_json

from pyspark.sql import SparkSession

def test_prep_for_linear_regression(spark: SparkSession):
    diamonds_df = create_df_from_json("tests/fixtures/diamonds.json", spark)

    actual_df = transform(diamonds_df)
    assert data_frame_to_json(actual_df) == expected_json()

.                                                                                            [100%]


# Make The Assert Specific

Right now, our test compares everything in the output dataframe to everything in a large json file. That's a lot of rows to compare and the assert is wrong anyways!

Let's make this test assert on the thing we actually care about - the output price of the diamond!

## Let's make our assert specific!
### We can do the next step in one of 3 ways:
#### With Chispa
- [ ] Add these imports to the top of the cell, below the `%%ipytest -qq` line:  
`from chispa import assert_column_equality`  
`from pySpark.sql import functions`
- [ ] Filter the dataframe for the unique id of the diamond we care about:  
`actual_df=actual_df.filter(actual_df.id == 'DI-26-null-price')`
- [ ] Create a new column in our dataframe that contains our expected price:  
`actual_df=actual_df.withColumn('expected_price', lit(2960.0))`
- [ ] Assert the value in the price column matches the value we want:  
`assert_column_equality(actual_df, 'price', 'expected_price')`


In [ ]:
%%ipytest -qq
from src.linear_regression_prep import transform
from tests.test_helpers.json_helpers import create_df_from_json, data_frame_to_json

from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from chispa import assert_column_equality

def test_prep_for_linear_regression(spark: SparkSession):
    diamonds_df = create_df_from_json("tests/fixtures/diamonds.json", spark)

    actual_df = transform(diamonds_df)


.                                                                                            [100%]


### With Pandas
- [ ] import pandas:  
`import pandas as pd`
- [ ] Filter the dataframe for the unique id of the diamond we care about:  
  `actual_df=actual_df.filter(actual_df.id == 'DI-26-null-price')`
- [ ] Create your expected dataframe using Pandas:  
 `expected = pd.DataFrame(({'id': ["DI-26-null-price"], 'price':[2690.0] }))`
- [ ] Select the column you care about:  
  `actual_df=actual_df.select(['id', 'price'])
- [ ] Assert for dataframe equality using pandas:  
  `pd.testing.assert_frame_equal(actual_df, expected)`

In [13]:
%%ipytest -qq
from src.linear_regression_prep import transform
from tests.test_helpers.json_helpers import create_df_from_json, data_frame_to_json

from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from chispa import assert_column_equality

import pandas as pd

def test_prep_for_linear_regression(spark: SparkSession):
    diamonds_df = create_df_from_json("tests/fixtures/diamonds.json", spark)

    actual_df = transform(diamonds_df)


F                                                                                            [100%]
============================================= FAILURES =============================================
_________________________________ test_prep_for_linear_regression __________________________________

spark = <pyspark.sql.session.SparkSession object at 0x7f09a86a93f0>

    def test_prep_for_linear_regression(spark: SparkSession):
        diamonds_df = create_df_from_json("tests/fixtures/diamonds.json", spark)
    
        actual_df = transform(diamonds_df)
        actual_df=actual_df.filter(actual_df.id == 'DI-26-null-price')
        expected = pd.DataFrame(({'id': ["DI-26-null-price"], 'price':[2690.0] }))
        actual_df = actual_df.select(['id', 'price']).where(actual_df.id == "DI-26-null-price")
>       pd.testing.assert_frame_equal(actual_df.toPandas(), expected)

<ipython-input-13-0be942c8d32b>:17: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _

### Assert on properties

- [ ] Filter the dataframe for the unique id of the diamond we care about:  
  `actual_df=actual_df.filter(actual_df.id == 'DI-26-null-price')`
- [ ] Convert your dataframe to JSON:  
`actual_df_json = data_frame_to_json(actual_df)`
- [ ] Assert the price property of the first object matches your expected price:  
`assert actual_df_json[0]['price'] == 2690.0`

In [14]:
%%ipytest -qq
from src.linear_regression_prep import transform
from tests.test_helpers.json_helpers import create_df_from_json, data_frame_to_json

from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from chispa import assert_column_equality

def test_prep_for_linear_regression(spark: SparkSession):
    diamonds_df = create_df_from_json("tests/fixtures/diamonds.json", spark)

    actual_df = transform(diamonds_df)



F                                                                                            [100%]
============================================= FAILURES =============================================
_________________________________ test_prep_for_linear_regression __________________________________

spark = <pyspark.sql.session.SparkSession object at 0x7f09a86c1c60>

    def test_prep_for_linear_regression(spark: SparkSession):
        diamonds_df = create_df_from_json("tests/fixtures/diamonds.json", spark)
    
        actual_df = transform(diamonds_df)
    
        actual_df=actual_df.filter(actual_df.id == 'DI-26-null-price')
        actual_df_json = data_frame_to_json(actual_df)
>       assert actual_df_json[0]['price'] == 2690.0
E       assert 2460.0 == 2690.0

<ipython-input-14-e71ba456c136>:15: AssertionError
--------------------------------------- Captured stdout call ---------------------------------------
[{'id': 'DI-26-null-price', 'carat': 0.21, 'clarity': 'VVS1', 'color'